# Crawling Data

## Twitter

In [ ]:
!pip install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

In [ ]:
def tweet_scrape(tag, count=100):
  list_tweet = []
  for i,tweet in enumerate(sntwitter.TwitterHashtagScraper(tag).get_items()):
    if i > count:
      break
    list_tweet.append({
      'Nama': tweet.user.username, 
      'Content': tweet.content, 
      'Lokasi': tweet.user.location
    })
  return pd.DataFrame(list_tweet) 

In [ ]:
tweet = tweet_scrape('capres2024 AND ganjarpranowo')
tweet.to_csv('ganjarScrape.csv', index=False)

<ipython-input-22-cbc57a5e4014>:8: FutureWarning: content is deprecated, use rawContent instead
  'Content': tweet.content,


## Web

In [ ]:
!pip install bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests as req

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
import pandas as pd

### pta.trunojoyo.ac.id

In [ ]:
class ScrapingPtaTrunojoyo:
  def __init__(self, url):
    self.url = url
  
  def page(self):
    res = req.get(self.url)
    soup = BeautifulSoup(res.content, 'html.parser')
    contents = soup.find('ol', class_='pagination').findChildren(recursive=False)
    return int(contents[-3].text)

  def artikel_page(self, page):
    res = req.get(f'{self.url}/{page}')
    soup = BeautifulSoup(res.content, 'html.parser')
    contents = soup.find_all('li', attrs={'data-cat':'#luxury'})
    return contents

  def get_berita(self):
    pta_list = []
    page = self.page()
    for pg in range(1, page+1):
      artikels = self.artikel_page(pg)
      for artikel in artikels:
        respone = req.get(artikel.find('a', class_='gray button')['href'])
        soup_data = BeautifulSoup(respone.content, 'html.parser')
        cont = soup_data.find('div', attrs={'id':'content_journal'})
        # ambil data
        nama = cont.findAll('div')[1].div.span.text.split(' : ')[1]
        judul = str.join("", (cont.find('a', class_='title').string).splitlines()).capitalize()
        abstrak = str.join("", (cont.find('p').string).splitlines()).capitalize()
        pta_list.append({
            'Penulis' : nama,
            'Judul': judul,
            'Abstrak': abstrak
        })
    self.pta = pta_list
  
  def result(self):
    return pd.DataFrame(self.pta)
  
  def save(self, namefile='pta'):
    df = pd.DataFrame(self.pta).to_csv(namefile+'.csv', index=False)
    print('Berhasil di save')

In [ ]:
scPta = ScrapingPtaTrunojoyo('https://pta.trunojoyo.ac.id/c_search/byfac/4')

In [ ]:
scPta.get_berita()

In [ ]:
scPta.result()

,Penulis,Judul,Abstrak
0,A.Ubaidillah S.Kom,Perancangan dan implementasi sistem database t...,Sistem informasi akademik (siakad) merupaka...
1,"M. Basith Ardianto,",Aplikasi kontrol dan monitoring jaringan kompu...,Berjalannya koneksi jaringan komputer dengan l...
2,"Akhmad Suyandi, S.Kom",Rancang bangun aplikasi proxy server untukenkr...,Web server adalah sebuah perangkat lunak serve...
3,Heri Supriyanto,Sistem pendukung keputusan optimasi penjadwala...,Penjadwalan kuliah di perguruan tinggi me...
4,Septian Rahman Hakim,Sistem augmented reality animasi benda bergera...,Seiring perkembangan teknologi yang ada diduni...
5,Adi Chandra Laksono,Gerak pekerja pada game real time strategy men...,Gerak pekerja ada pada game yang memiliki genr...
6,NURRACHMAT,Rancang bangun game perawatan sapi karapan men...,"Perkembangan game yang semakin pesat, memberik..."
7,Muhammad Choirur Rozi,Ekstraksi fitur berbasis two dimensional linea...,Sistem pengenalan wajah adalah suatu sistem un...
8,M Khoiril Anwar,Implementasi algoritma prim dan depth first ...,Teknologi mobile game beroperating system open...
9,Elky Lilik Misdayani,Sistem informasi perusahaan otobus sumber kenc...,Sistem informasi pengolahan data yang dibutuhk...


In [ ]:
scPta.save()

Berhasil di save


In [ ]:
class ScrapingKompas:
  def __init__(self, url):
    self.url = url

  def page(self):
    res_ = req.get(self.url)
    soup_ = BeautifulSoup(res_.content, 'html.parser')
    contents_ = soup_.findAll('a', class_='paging__link')[:-2]
    return len(contents_)

  def artikel_page(self, page):
    res_ = req.get(f'{self.url}?page={page}')
    soup_ = BeautifulSoup(res_.content, 'html.parser')
    contents_ = soup_.findAll('a', class_='article__link')
    return contents_

  def get_berita(self):
    berita_list = []
    page = self.page()
    for pg in range(1, page+1):
      artikels = self.artikel_page(pg)
      for artikel in artikels:
        res = req.get(artikel['href'])
        soup = BeautifulSoup(res.content, 'html.parser')
        # ambil data
        judul = soup.find('h1', class_='read__title').string
        isi = isi = ' '.join(soup.find('div', class_='read__content').stripped_strings)
        berita_list.append({
            'Judul': judul,
            'Isi Berita': isi
        })
    self.beritas = berita_list
    return 
  
  def result(self):
    return pd.DataFrame(self.beritas)
  
  def save(self, namefile='beritaKompas'):
    df = pd.DataFrame(self.beritas).to_csv(namefile+'.csv', index=False)
    print('Berhasil di save')

In [ ]:
data = ScrapingKompas('https://indeks.kompas.com/?site=nasional')

In [ ]:
data.get_berita()

In [ ]:
data.result()

,Judul,Isi Berita
0,"Hari Ini dalam Sejarah, Raja Faisal dari Arab ...",KOMPAS.com — Faisal bin Abdulaziz al-Saud dike...
1,"Tarif MRT Bundaran HI-Lebak Bulus Rp 8.500, LR...","JAKARTA, KOMPAS.com - Dalam rapat pimpinan gab..."
2,"MUI Minta Masyarakat Stop Pakai Istilah ""Cebon...","Jakarta, KOMPAS.com — Majelis Ulama Indonesia ..."
3,Pemantau Asing dari 33 Negara Akan Ikut Memoni...,"JAKARTA, KOMPAS.com — Komisioner Komisi Pemili..."
4,Ridwan Kamil Optimistis Tol Cisumdawu Bawa Per...,"BANDUNG , KOMPAS.com - Perkembangan pengerjaan..."
5,5 Fakta Turis Asing Ingin Dekati Kawah hingga ...,KOMPAS.com - Seorang wisatawan asing di kawasa...
6,TKN Jokowi-Ma'ruf Sebut Ada Bendera HTI di Kam...,"JAKARTA, KOMPAS.com - Anggota Dewan Pengarah B..."
7,Cemburu Jadi Motif Pelaku Bunuh Pria yang Jena...,"BATAM , KOMPAS.com - Kasus pembunuhan pria yan..."
8,Video: Detik-detik Jatuhnya Meteor 10 Kali Bom...,"KOMPAS.com - 18 Desember tahun lalu, sebuah me..."
9,Survei Charta Politika: Elektabilitas PSI Tert...,JAKARTA KOMPAS.com - Survei terbaru Charta Pol...


In [ ]:
data.save()

## Web

In [1]:
!pip install requests bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1270 sha256=d15086cbfe01ea9f7e2d90d3560a843339a1d58a06668cd44c1f9c8fea767940
  Stored in directory: /root/.cache/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4


In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### pta.trunojoyo.ac.id

In [10]:
pta = []

for i in range(1,172):
  URL = "https://pta.trunojoyo.ac.id/c_search/byprod/10/" + str(i+1)
  r = requests.get(URL)
  soup_data = BeautifulSoup(r.text, 'html.parser')
  items = soup_data.find('div', {'id':'content_journal'})

  for it in items.findAll('li',{'data-cat':'#luxury'}):
    namaPenulis = it.span.text.strip('\n').replace('Penulis : ', '')
    
    link = it.find('a', 'gray button')['href']
    req = requests.get(link)
    soup_data2 = BeautifulSoup(req.text, 'html.parser')
    abstraks = soup_data2.findAll('div', {'style':'margin: 15px 15px 15px 15px;'})
    abstrak = soup_data2.find('p', {'align':'justify'}).text
    nim = link.replace('https://pta.trunojoyo.ac.id/welcome/detail/', '')

    pta.append([namaPenulis, nim, it.a.text.strip('\n'), abstrak])
    
pta_df = pd.DataFrame(pta, columns=['Penulis', 'NIM', 'Judul', 'Abstrak'])
pta_df

,Penulis,NIM,Judul,Abstrak
0,Adi Chandra Laksono,070411100007,Gerak Pekerja Pada Game Real Time Strategy Men...,Gerak pekerja ada pada game yang memiliki genr...
1,NURRACHMAT,070411100126,RANCANG BANGUN GAME PERAWATAN SAPI KARAPAN MEN...,"Perkembangan game yang semakin pesat, memberik..."
2,Muhammad Choirur Rozi,070411100109,EKSTRAKSI FITUR BERBASIS TWO DIMENSIONAL LINEA...,Sistem pengenalan wajah adalah suatu sistem un...
3,M Khoiril Anwar,080411100083,IMPLEMENTASI ALGORITMA PRIM DAN DEPTH FIRST ...,Teknologi mobile game beroperating system open...
4,MALIKUL HAMZAH,070411100092,Perancangan Sistem Informasi Badan Kepegawaian...,Kantor Badan Kepegawaian kota Bangkalan adalah...
...,...,...,...,...
848,Rachmad Agung Pambudi,160411100032,PENERAPAN ALGORITMA LONG-SHORT TERM MEMORY UNT...,Investasi saham selama ini memiliki resiko ker...
849,Nadila Hidayanti,160411100182,SISTEM PENCARIAN TEKS AL-QURAN TERJEMAHAN BERB...,Information Retrieval (IR) merupakan pengambil...
850,Afni Sakinah,160411100077,KLASIFIKASI KOMPLEKSITAS VISUAL CITRA SAMPAH M...,Klasifikasi citra merupakan proses pengelompok...
851,Friska Fatmawatiningrum,160411100084,IDENTIFIKASI BINER ATRIBUT PEJALAN KAKI MENGGU...,Identifikasi atribut pejalan kaki merupakan sa...


In [19]:
pta_df.to_csv('pta.csv', index=False)

### Kompas

In [ ]:
kompas = []

URL = "https://indeks.kompas.com/?site=nasional"
r = requests.get(URL) 
soup_data = BeautifulSoup(r.text, 'html.parser')
items = soup_data.find('a', 'main-left')

for it in items.findAll('h1','title'):
  judul = it.a.text
  link = it.a['href']

  req = requests.get(link)
    
  soup2 = BeautifulSoup(req.text, 'html.parser')
  contents = soup2.find('div', 'detail-in').text.strip('\nTEMPO.CO,')

  kompas.append([judul, contents])
    
kompas_df = pd.DataFrame(kompas, columns=['Judul', 'Isi Berita'])
kompas_df